Adapted From: https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb#scrollTo=mjJGEXShp7te

In [ ]:
!pip install accelerate -U
# !pip install -q datasets

In [ ]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from transformers import BertTokenizer, BertModel
from torch import nn
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import pickle
# from datasets import load_dataset

In [ ]:
classnames = ["mistral", "rag"]
num_labels=len(classnames)
batch_size = 16

In [ ]:
with open("labeled_data.pkl", "rb") as f:
  labeled_data = pickle.load(f)

In [ ]:
labeled_data = filter(lambda x: x["labels"] in ["mistral", "rag"], labeled_data)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Custom Model
def encode(data):
  text_lower = data["question"].lower()
  label = data["labels"]
  encoded_text = tokenizer(text_lower,
                            padding='max_length',
                            truncation=True,
                            max_length=256,
                            return_tensors="pt"
                            )
  # set unlabeled to most difficult level
  if label not in classnames:
    label = classnames[-1]

  encoded_text['labels'] = torch.LongTensor([classnames.index(label)])
  return encoded_text

# Use this for HF trainer
def encode_aslist(data):
  text_lower = data["question"].lower()
  label = data["labels"]
  encoded_text = tokenizer(text_lower,
                            padding='max_length',
                            truncation=True,
                            max_length=128,
                            # return_tensors="pt"
                            )
  if label not in classnames:
    label = classnames[-1]

  encoded_text['labels'] = [classnames.index(label)]

In [ ]:
encoded_data = list(map(encode_aslist, labeled_data))

In [ ]:
print(len(encoded_data))

1000


# Using BERTForSequenceClassification + HF Trainer

In [ ]:
import numpy as np

def get_accuracy(output):
  pred = np.argmax(output.predictions, axis=1)
  labels = output.label_ids
  accuracy = (pred[:, np.newaxis] == labels).sum() / pred.shape[0]
  return {"accuracy": accuracy}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="single_label_classification",
                                                           num_labels=num_labels,
                                                           )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for param in model.base_model.parameters():
  param.requires_grad = False

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-hotpotqa-classifier-2",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [ ]:
torch.random.manual_seed(42)
train_set, val_set = torch.utils.data.random_split(list(encoded_data), [0.85, 0.15])

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    compute_metrics=get_accuracy
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.711949,0.593333
2,No log,0.654993,0.646667
3,No log,0.608947,0.733333
4,No log,0.605878,0.726667
5,No log,0.611789,0.733333


TrainOutput(global_step=270, training_loss=0.5757288614908854, metrics={'train_runtime': 142.6393, 'train_samples_per_second': 29.795, 'train_steps_per_second': 1.893, 'total_flos': 279558006336000.0, 'train_loss': 0.5757288614908854, 'epoch': 5.0})

# Custom Model (performed worse than default BERT Classifier)

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=batch_size)
val_dataloader = DataLoader(val_set, batch_size=batch_size)

In [ ]:
next(iter(train_dataloader))

In [ ]:
class BertQuestionClassifier(nn.Module):

  def __init__(self, hidden_dim=768, device=None):
    super(BertQuestionClassifier, self).__init__()

    if device is None:
      self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
    else:
      self.device = device

    self.bert = BertModel.from_pretrained('bert-base-uncased').to(self.device)
    self.classifier = nn.Sequential(
      nn.Linear(hidden_dim, hidden_dim),
      nn.ReLU(),
      # nn.Dropout(0.1),
      nn.Linear(hidden_dim, hidden_dim),
      nn.ReLU(),
      # nn.Dropout(0.1),
      nn.Linear(hidden_dim, 2),
    ).to(self.device)

  def forward(self, input_ids, attention_mask=None, labels=None, token_type_ids=None):
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = outputs[1]
    logits = self.classifier(sequence_output)
    return logits


In [ ]:
model = BertQuestionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [ ]:
for param in model.bert.parameters():
  param.requires_grad = False

In [ ]:
from tqdm import tqdm

def train_single_epoch():
  n_examples = len(train_set)
  model.train()
  total_loss = 0
  total_accuracy = 0
  for i ,batch in tqdm(enumerate(train_dataloader)):
    #print([v[0] for k,v in batch.items()])
    batch = {k: v.to(model.device).squeeze(1) for k,v in batch.items()}
    logits = model(**batch)
    loss = criterion(logits, batch["labels"])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss += loss.sum().item()
    total_accuracy += (logits.argmax(dim=1) == batch["labels"]).sum().item()
  return total_loss / n_examples, total_accuracy / n_examples


def eval_once():
  n_examples = len(val_set)
  model.eval()
  total_loss = 0
  total_accuracy = 0
  for i ,batch in tqdm(enumerate(val_dataloader)):
    batch = {k: v.to(model.device).squeeze(1) for k,v in batch.items()}
    logits = model(**batch)
    loss = criterion(logits, batch["labels"])
    total_loss += loss.sum().item()
    total_accuracy += (logits.argmax(dim=1) == batch["labels"]).sum().item()
  return total_loss / n_examples, total_accuracy / n_examples


In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
for k, v in batch.items():
  v = torch.vstack(v).transpose(1,0)
  print(k, v.shape)

In [ ]:
best_acc = 0.0
for epoch in range(100):
  print(epoch, "train (loss, acc)", train_single_epoch())
  print(epoch, "val (loss, acc)", eval_once())